In [19]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")
from lifelines import CoxPHFitter
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [20]:
import os
os.chdir('C:\\0-ece204\\pr5')

In [21]:
full_data = pd.read_csv('Full Data.txt', index_col = [0])
full_data

,gender,ajcc_pathologic_tumor_stage,tumor_status,DSS,DSS.time,TP53,TTN,FAT1,MUC16,CSMD3,...,NIPBL,CEACAM5,CEACAM6,CLCA4,EGFR,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,Splice_Site,age
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,1,0,0,0.0,0.279452,822.678,1157.3300,1620.71,1004.4700,1.2325,...,1595.440,56.0776,423.3550,502.2340,722.847,92,8,0,2,1
TCGA-BA-4074,1,0,1,1.0,1.265753,1805.510,103.6870,6415.45,17.3422,0.3854,...,1605.890,153.3820,275.9330,12.7176,1267.900,106,8,0,1,1
TCGA-BA-4075,1,1,1,1.0,0.775342,383.760,295.9180,4162.40,1.0720,36.9824,...,907.410,18.2232,45.5581,10.1836,4833.430,113,10,0,4,0
TCGA-BA-4077,0,0,1,1.0,3.106849,1929.260,207.3870,7246.87,432.6270,0.0000,...,1174.690,2392.6900,2437.9200,720.8850,2371.940,234,20,0,6,0
TCGA-BA-5149,1,0,1,1.0,2.208219,165.298,347.9540,8848.73,110.5410,0.3422,...,1798.770,37.3032,447.6390,5.4757,6934.280,96,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-UF-A7JO,0,0,1,1.0,1.728767,1604.880,176.1930,1881.36,347.7040,2.8008,...,854.256,1498.8500,2014.6000,82.8248,530.911,135,9,0,4,1
TCGA-UF-A7JS,1,0,1,1.0,1.863014,2457.400,1379.4300,1339.31,243.6470,19.4320,...,982.063,358.7440,294.4690,32.8849,246.637,50,1,0,1,1
TCGA-UF-A7JT,0,0,1,1.0,2.720548,1684.850,149.5190,3430.74,213.8530,14.2857,...,1009.520,49.7835,209.5240,200.4330,790.905,152,10,0,3,1


# Cox model

In [22]:
def coxfitter(data, duration, event):
    # Define the CoxPHFitter object
    cph = CoxPHFitter()
    # Fit the model to your data
    cph.fit(data, duration_col= duration, event_col=event)
    summary = cph.summary
    return summary

In [23]:
def cox_model(df, time, lis):
    data = df[df['DSS.time'] <= time]
    signi = []
    pvals = []
    pvals_correct = []
    for i in lis:
        summary = coxfitter(data[[i ,'gender','ajcc_pathologic_tumor_stage', 
                                   'tumor_status', 'age', 'DSS.time', 'DSS']], 'DSS.time', 'DSS')
        p_values = summary["p"]
        rejected, pvals_corrected = multipletests(p_values, method='fdr_bh')[:2]
        summary['p_corrected'] = pvals_corrected
        p = summary.loc[i, "p"]
        p_correct = summary.loc[i, "p_corrected"]
        pvals.append(p)
        pvals_correct.append(p_correct)
        signi.append(rejected[0])
    features = [col for col in lis]  # list of features
    # Create a DataFrame with the results
    results = pd.DataFrame({'Features': features,'p': pvals,'p_corrected': pvals_correct, 'significant': signi})
    return results[results['significant'] == True]


In [24]:
# mutation
mutation = ['Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'Splice_Site']
cox_model(full_data, 2, mutation)# example: use full data and 2 years as limit

,Features,p,p_corrected,significant
0,Missense_Mutation,4.747304e-08,2.373652e-07,True
1,Nonsense_Mutation,3.141682e-04,1.570841e-03,True
2,Nonstop_Mutation,8.020092e-03,4.010046e-02,True
3,Splice_Site,3.520589e-05,1.760294e-04,True


In [27]:
# feature dic for mutation
time_limit = [1, 2, 5, 7, 10]
feature_dic = {}
for i in time_limit:
    df = cox_model(full_data, i, mutation)
    feature_dic[i] = list(df['Features'])
feature_dic

{1: ['Missense_Mutation', 'Nonsense_Mutation'],
 2: ['Missense_Mutation',
  'Nonsense_Mutation',
  'Nonstop_Mutation',
  'Splice_Site'],
 5: ['Missense_Mutation',
  'Nonsense_Mutation',
  'Nonstop_Mutation',
  'Splice_Site'],
 7: ['Missense_Mutation', 'Nonsense_Mutation', 'Splice_Site'],
 10: ['Missense_Mutation', 'Nonsense_Mutation', 'Splice_Site']}

# Classification

In [28]:
#select top n mutations by z values as features columns from full_data

In [29]:
def classifcation_svm(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # SVM
   
    # Define the model
    svm_clf = svm.SVC()

    # Train the model
    svm_clf.fit(X_train, y_train)
   
    # Predict on the test set
    svm_predictions = svm_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, svm_predictions)
    #print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

In [30]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_svm(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('SVM Accuracy ' + str(mean_acc))

For time limit 1 and gene expression features from Cox model
SVM Accuracy 0.7059177777777705
For time limit 2 and gene expression features from Cox model
SVM Accuracy 0.7068744444444389
For time limit 5 and gene expression features from Cox model
SVM Accuracy 0.7065288888888785
For time limit 7 and gene expression features from Cox model
SVM Accuracy 0.705873333333329
For time limit 10 and gene expression features from Cox model
SVM Accuracy 0.7050222222222176


In [31]:
def classifcation_randomforest(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_clf = RandomForestClassifier()

    # Train the model
    rf_clf.fit(X_train, y_train)
    # Predict on the test set
    rf_predictions = rf_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, rf_predictions)
    #print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))

In [32]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_randomforest(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('Random Forest Accuracy ' + str(mean_acc))

For time limit 1 and gene expression features from Cox model
Random Forest Accuracy 0.7002222222222161
For time limit 2 and gene expression features from Cox model
Random Forest Accuracy 0.7305777777777784
For time limit 5 and gene expression features from Cox model
Random Forest Accuracy 0.7305911111111112
For time limit 7 and gene expression features from Cox model
Random Forest Accuracy 0.7278144444444412
For time limit 10 and gene expression features from Cox model
Random Forest Accuracy 0.7277699999999974


In [33]:
def classifcation_logistic(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    log_reg = LogisticRegression()

    # Train the model
    log_reg.fit(X_train, y_train)
    log_reg_predictions = log_reg.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, log_reg_predictions)
    #print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_predictions))

In [34]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_logistic(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('Logistic Regression Accuracy ' + str(mean_acc))

For time limit 1 and gene expression features from Cox model
Logistic Regression Accuracy 0.7018211111111042
For time limit 2 and gene expression features from Cox model
Logistic Regression Accuracy 0.705785555555551
For time limit 5 and gene expression features from Cox model
Logistic Regression Accuracy 0.7049799999999916
For time limit 7 and gene expression features from Cox model
Logistic Regression Accuracy 0.7039277777777715
For time limit 10 and gene expression features from Cox model
Logistic Regression Accuracy 0.7028322222222135
